In [12]:
import os
import re
import nltk
import numpy as np
import pandas as pd

import config
import sys
print(sys.version)


3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]


In [22]:

assert os.path.exists(config.conceptnet_path)
conceptnet_df = pd.read_csv(config.conceptnet_path, names=["uri", "relation", "snode", "enode", "more"], sep="\t")


In [2]:

START_ID = '<START_ID>'
END_ID = '<END_ID>'
PAD_ID = '<PAD_ID>'
UNK_ID = '<UNK_ID>'

import tensorlayer as tl

vocab = tl.nlp.Vocabulary(
    config.news20_vocab_path, start_word=START_ID, end_word=END_ID, unk_word=UNK_ID
)

INFO:tensorflow:Initializing vocabulary from file: ../data/20-newsgroups/clean/vocab.txt


[TL] Vocabulary from ../data/20-newsgroups/clean/vocab.txt : <START_ID> <END_ID> <UNK_ID>


[TL]     vocabulary with 23568 words (includes start_word, end_word, unk_word)


[TL]       start_id: 35


[TL]       end_id  : 34


In [17]:
# print(df.loc[0])
ndf = df[df["snode"] == "/c/en/%s" % "atheism"]
print(ndf[["relation", "snode", "enode"]])
ndf = df[df["enode"] == "/c/en/%s" % "atheism"]
print(ndf[["relation", "snode", "enode"]])


             relation          snode                   enode
90911    /r/DefinedAs  /c/en/atheism     /c/en/denial_of_god
90912    /r/DefinedAs  /c/en/atheism  /c/en/disbelief_in_god
951697         /r/IsA  /c/en/atheism            /c/en/belief
1301118  /r/RelatedTo  /c/en/atheism       /c/en/agnosticism
1301119  /r/RelatedTo  /c/en/atheism            /c/en/atheal
1301120  /r/RelatedTo  /c/en/atheism            /c/en/athean
1301121  /r/RelatedTo  /c/en/atheism           /c/en/atheism
1301122  /r/RelatedTo  /c/en/atheism         /c/en/atheonism
1301123  /r/RelatedTo  /c/en/atheism             /c/en/deism
1301124  /r/RelatedTo  /c/en/atheism        /c/en/monotheism
1301125  /r/RelatedTo  /c/en/atheism       /c/en/panentheism
1301126  /r/RelatedTo  /c/en/atheism         /c/en/pantheism
1301127  /r/RelatedTo  /c/en/atheism        /c/en/polytheism
1301128  /r/RelatedTo  /c/en/atheism          /c/en/religion
1301129  /r/RelatedTo  /c/en/atheism            /c/en/theism
2841940    /r/Synonym  /

                           relation                         snode  \
14075                    /r/Antonym                /c/en/religion   
77613                     /r/Causes  /c/en/learning_about_science   
203063               /r/DerivedFrom              /c/en/panatheism   
277624   /r/EtymologicallyRelatedTo                 /c/en/adevism   
278468   /r/EtymologicallyRelatedTo            /c/en/apeirotheism   
324837                    /r/FormOf              /c/en/atheisms/n   
324878                    /r/FormOf               /c/en/athiesm/n   
1272212                /r/RelatedTo           /c/en/antiatheist/n   
1272214                /r/RelatedTo         /c/en/antiatheistic/a   
1275038                /r/RelatedTo       /c/en/antimetaphysics/n   
1279079                /r/RelatedTo               /c/en/apatheism   
1301121                /r/RelatedTo                 /c/en/atheism   
1301136                /r/RelatedTo              /c/en/atheisms/n   
1301137                /r/RelatedT

In [174]:

def preprocess(text):
    ntext = text.split("/")[3] if "/" in text else text
    ntext = re.sub(r'[\W_]+', ' ', ntext)
    textlist = ntext.split()
    textlist = [t for t in textlist if vocab.word_to_id(t) != vocab.unk_id and len(t) >= 3]
    
    if len(textlist) <= 1:
        return textlist
    else:
        text_with_tag = nltk.pos_tag(textlist)  # a list of words a list of words with part of speech
        selected = [text_tag[0] for text_tag in text_with_tag \
                    if text_tag[1] in config.pos_dict and \
                    (config.pos_dict[text_tag[1]] == 'a' or config.pos_dict[text_tag[1]] == 'n')]
        return selected

print(preprocess("/c/en/learning_about_science/n"))
print(preprocess("/c/en/atheism/n"))
print(preprocess("/c/en/atheism_a/n"))
print(preprocess("atheism"))
print(preprocess("/c/en/christan_science"))
print(preprocess("dkjflasdk"))


['science']
['atheism']
['atheism']
['atheism']
['science']
[]


In [ ]:

glove_dict = dict()
glove_filename = config.word_embed_file_path
with open(glove_filename, 'r') as f:
    for idx, line in enumerate(f):
        content = line.replace("\n", "").split(" ")

        word = content[0]
        vect = np.array(content[1:]).astype(np.float32)
        
        glove_dict[word] = vect

from scipy.spatial.distance import cosine

print(cosine(glove_dict["religion"], glove_dict["atheism"]))
print(cosine(glove_dict["religion"], glove_dict["science"]))
print(cosine(glove_dict["atheism"], glove_dict["science"]))
print(cosine(glove_dict["mac"], glove_dict["windows"]))


In [79]:
text_df = pd.read_csv(config.news20_train_path)

sample_text = preprocess(text_df["selected_tfidf"].loc[0])
print(sample_text[:])


['free', 'energy', 'technology', 'free', 'energy', 'technology', 'robert', 'mcelwaine', 'physicist', 'heavier', 'air', 'machine', 'laws', 'example', 'simon', 'demonstration', 'possible', 'known', 'substances', 'forms', 'machinery', 'forms', 'machine', 'long', 'distances', 'air', 'complete', 'possible', 'demonstration', 'physical', 'fact', 'such', 'wright', 'such', 'pronouncements', 'take', 'airplanes', 'physicists', 'other', 'things', 'several', 'free', 'energy', 'terms', 'such', 'pseudo', 'science', 'perpetual', 'motion', 'laws', 'energy', 'law', 'thermodynamics', 'energy', 'law', 'physicists', 'certain', 'things', 'things', 'such', 'common', 'orthodox', 'modern', 'science', 'cover', 'inconsistencies', 'contradictions', 'modern', 'theories', 'free', 'energy', 'devices', 'unlimited', 'supply', 'energy', 'universe', 'fuel', 'world', 'wide', 'energy', 'crisis', 'pollution', 'degradation', 'depletion', 'environment', 'free', 'energy', 'devices', 'energy', 'tap', 'energy', 'sources', 'form

In [52]:

positive_relation_set = set([
    '/r/IsA', '/rr/IsA', 
    '/r/PartOf', '/rr/PartOf', 
    '/r/AtLocation', '/rr/AtLocation', 
    '/r/RelatedTo', '/r/Causes',
    '/r/Synonym', '/r/CreatedBy',
    '/r/HasProperty', '/r/UsedFor'
])

negative_relation_set = set([
    '/r/Antonym', '/r/DistinctFrom',
    '/r/NotIsA', '/r/NotHasProperty',
    '/r/NotUsedFor'
])

print("/r/IsA" in positive_relation_set)
print("/r/NotIsA" in positive_relation_set)
print("/r/IsA" in negative_relation_set)
print("/r/NotIsA" in negative_relation_set)
print(type(positive_relation_set))
print(type(negative_relation_set))



['free', 'energy', 'technology', 'free', 'energy', 'technology', 'robert', 'mcelwaine', 'physicist', 'heavier', 'air', 'machine', 'laws', 'example', 'simon', 'demonstration', 'possible', 'known', 'substances', 'forms', 'machinery', 'forms', 'machine', 'long', 'distances', 'air', 'complete', 'possible', 'demonstration', 'physical', 'fact', 'such', 'wright', 'such', 'pronouncements', 'take', 'airplanes', 'physicists', 'other', 'things', 'several', 'free', 'energy', 'terms', 'such', 'pseudo', 'science', 'perpetual', 'motion', 'laws', 'energy', 'law', 'thermodynamics', 'energy', 'law', 'physicists', 'certain', 'things', 'things', 'such', 'common', 'orthodox', 'modern', 'science', 'cover', 'inconsistencies', 'contradictions', 'modern', 'theories', 'free', 'energy', 'devices', 'unlimited', 'supply', 'energy', 'universe', 'fuel', 'world', 'wide', 'energy', 'crisis', 'pollution', 'degradation', 'depletion', 'environment', 'free', 'energy', 'devices', 'energy', 'tap', 'energy', 'sources', 'form

In [121]:
def remove_word(word):
    pword = preprocess(word)
    pword = ' '.join(pword)
    if vocab.word_to_id(pword) == vocab.unk_id or len(pword) < 3:
        return np.nan
    else:
        return pword

def binary_relation(relation):
    if relation in positive_relation_set:
        return True
    elif relation in negative_relation_set:
        return False
    else:
        return np.nan

def preprocess_conceptnet(df):
    ndf = df[["snode", "enode", "relation"]]
    ndf["relation"] = ndf["relation"].apply(lambda r: binary_relation(r))
    ndf = ndf.dropna()
    ndf = ndf.drop_duplicates()
    ndf["snode"] = ndf["snode"].apply(lambda w: remove_word(w))
    ndf["enode"] = ndf["enode"].apply(lambda w: remove_word(w))
    ndf = ndf.dropna()
    ndf = ndf.drop_duplicates()
    return ndf

# print(remove_word("/c/en/learning_about_science"))

In [122]:

processed_conceptnet_df = preprocess_conceptnet(conceptnet_df)


In [123]:
print(processed_conceptnet_df)



In [187]:

def BFS(nodelist, relation_df, max_hop):
    positive_node_dict = {"pending": set(), "complete": set()}
    negative_node_dict = {"pending": set(), "complete": set()}
    neutral_node_dict = {"pending": set(), "complete": set()}
    controversial_node_dict = {"pending": set(), "complete": set()}
    
    hop_dict = dict()
    
    def add_positive(word):
        if word in neutral_node_dict["pending"]:
            neutral_node_dict["pending"].remove(word)
            positive_node_dict["pending"].add(word)
            assert word not in positive_node_dict["complete"]
        elif word in neutral_node_dict["complete"]:
            neutral_node_dict["complete"].remove(word)
            positive_node_dict["pending"].add(word)
            assert word not in positive_node_dict["complete"]
        elif word in negative_node_dict["pending"]:
            negative_node_dict["pending"].remove(word)
            controversial_node_set.add(word)
            # pass # do nothing
        elif word in negative_node_dict["complete"]:
            negative_node_dict["complete"].remove(word)
            controversial_node_set.add(word)
            # pass # do nothing
        elif word in positive_node_dict["pending"] or word in positive_node_dict["complete"]:
            pass # do nothing
        elif word in controversial_node_dict["pending"] or word in controversial_node_dict["complete"]:
            pass # do nothing
        else:
            positive_node_dict["pending"].add(word)
    
    def add_negative(word):
        if word in neutral_node_dict["pending"]:
            neutral_node_dict["pending"].remove(word)
            negative_node_dict["pending"].add(word)
            assert word not in negative_node_dict["complete"]
        elif word in neutral_node_dict["complete"]:
            neutral_node_dict["complete"].remove(word)
            negative_node_dict["pending"].add(word)
            assert word not in negative_node_dict["complete"]
        elif word in negative_node_dict["pending"] or word in negative_node_dict["complete"]:
            pass # do nothing
        elif word in positive_node_dict["pending"]:
            positive_node_dict["pending"].remove(word)
            # negative_node_dict["pending"].add(word)
            # controversial_node_set.add(word)
        elif word in positive_node_dict["complete"]:
            positive_node_dict["complete"].remove(word)
            negative_node_dict["pending"].add(word)
            # controversial_node_set.add(word)
        elif word in controversial_node_dict["pending"] or word in controversial_node_dict["complete"]:
            pass # do nothing
        else:
            negative_node_dict["pending"].add(word)
    
    def add_neutral(word):
        if word in neutral_node_dict["pending"] or word in neutral_node_dict["complete"]:
            pass # do nothing
        elif word in negative_node_dict["pending"] or word in negative_node_dict["complete"]:
            pass # do nothing
        elif word in positive_node_dict["pending"] or word in positive_node_dict["complete"]:
            pass # do nothing
        elif word in controversial_node_dict["pending"] or word in controversial_node_dict["complete"]:
            pass # do nothing
        else:
            neutral_node_dict["pending"].add(word)
            
    for node in nodelist:
        positive_node_dict["pending"].add(node)
        hop_dict[node] = 0
    
    print("BFS ...")
    round = 0
    while len(positive_node_dict["pending"]) + len(negative_node_dict["pending"])> 0:
        
        round += 1
        if round % 200 == 0:
            print(round, len(positive_node_dict["pending"]), len(negative_node_dict["pending"]))
        
        if len(positive_node_dict["pending"]) > 0:
            current_node = positive_node_dict["pending"].pop() 
            current_type = True
            positive_node_dict["complete"].add(current_node)
        else:
            current_node = negative_node_dict["pending"].pop()
            current_type = False
            negative_node_dict["complete"].add(current_node)

        current_hop = hop_dict[current_node] + 1
        if current_hop > max_hop:
            continue
        
        df1 = relation_df[relation_df["snode"] == current_node]
        df2 = relation_df[relation_df["enode"] == current_node]
        current_df = pd.concat([df1, df2])
        current_positive_relation_df = current_df[current_df["relation"] == True]
        current_negative_relation_df = current_df[current_df["relation"] == False]
        
        for idx, row in current_positive_relation_df.iterrows():
            node = row["snode"] if row["snode"] != current_node else row["enode"]
            if node == current_node:
                continue
            assert node != current_node
            
            if current_type: # current node is from positive_set
                add_positive(node)
            else: # current node is from negative_set
                add_negative(node)
                
            hop_dict[node] = current_hop
            
        for idx, row in current_negative_relation_df.iterrows():
            node = row["snode"] if row["snode"] != current_node else row["enode"]
            if node == current_node:
                continue
            assert node != current_node
            if current_type: # current node is from positive_set
                add_negative(node)
            else: # current node is from negative_set
                add_neutral(node)
                
            hop_dict[node] = current_hop
        
    
    assert len(positive_node_dict["pending"]) == 0
    assert len(negative_node_dict["pending"]) == 0
    assert len(neutral_node_dict["pending"]) == 0
    assert len(controversial_node_dict["pending"]) == 0
    
    positive_node_set = positive_node_dict["complete"]
    negative_node_set = negative_node_dict["complete"]
    neutral_node_set = neutral_node_dict["complete"]
    controversial_node_set = controversial_node_dict["complete"]
    
    return positive_node_set, negative_node_set, neutral_node_set, controversial_node_set
        
            
       


In [176]:
     
# text_df = pd.read_csv(config.news20_train_path)
# sample_text = preprocess(text_df["selected_tfidf"].loc[0])
pos_set, neg_set, neu_set, cont_set = BFS(["atheism"], processed_conceptnet_df, max_hop=3)
# pos_set, neg_set, neu_set, cont_set = BFS(["religion"], processed_conceptnet_df, max_hop=3)


In [186]:
# print(sorted(pos_set))
print(sorted(neg_set))
print(sorted(neu_set))
print(sorted(cont_set))
# BFS(["religion"], processed_conceptnet_df)
